# Simple Model Cavity **Gridap version**

Writen by: Oscar Reula (oreula@gmail.com)

This code is used to compute the resonant frequencies and eigenvectors of a cavity. The model is 2-dimensional, that is it corresponds to infinite long structures in the  ẑ direction. 
This restriction is mainly due to have a code that can run in home computers in very short time.

First the theory: we need to solve the following eigenfuntion problem: 

$$
\nabla \cdot \nabla u  = \lambda u \quad\quad\text{ in }\Omega
$$

$$u = 0 \text{ on } \;\;\;\partial\Omega
$$

Where $\Omega$ is some 2-dimensional region. 

In order to use finite elements we need to express the problem as a weak one:

$$
\int_{\Omega} \nabla u\,\cdot \nabla v\, {\rm d} x
= \lambda \int_{\Omega} u \cdot v\, {\rm d} x \quad \forall  v \in H^1_0(\Omega),
$$



We shall incorporate a simple, known, case which we shall use to test the implementation, namely:

$\nabla \cdot \nabla u  = \lambda u \quad\quad\text{ in } \Omega \quad\quad\quad
\Omega = (0,\pi)\times(0,\pi)$

For this case we know the solutions: 

$u_{n,m}(x,y) = \sin(n x)\sin(m y), \quad \lambda = m^2 + n^2,$ $\quad \lambda = 2, 5, 5, 8, 10, 10, 13, 13, 17, 17, 18, \dots$

We use this case to check that what we are doing is correct. 

Part of this code is addapted from: https://gist.github.com/Balaje/102485bb14ec6daf677f938fbd8f3ebb

### Status

The normal Dirichlet problem is working.

In [ ]:
using Gridap
using GridapMakie, GLMakie
GLMakie.activate!(inline=true)
using FileIO
#using Plots
#mkdir("models")
#mkdir("images")
using GridapGmsh
#import Gmsh: gmsh
using LinearAlgebra
using SparseArrays
using SuiteSparse
using Arpack

In [ ]:
dirichlet = true # Dirichlet boundary condition
neumann = false # Neumann boundary condition



test = false # test with no leaking, just to check the code.
rectangle = false # rectangle with leaking (parameter a)
square_circle = false
circle_circle = false
open_cavity = false # square with an interior cavity with an opening. 
                    # the cavity has no leaking except for the hole. 
                    # the outer part has Dirichlet.

#test = true
#rectangle = true
#square_circle = true
#circle_circle = true
open_cavity = true


In [ ]:
if test || rectangle 
    L=1.0
    asymmetry = 1.0 # si diferente de UNO da una asimetría y los autovalores dejan de ser degenerados.
    side_x = L*pi
    side_y = L*pi*asymmetry
    @show l = 50 #number of elements in the mesh
    @show domain = (0.0, side_x, 0.0, side_y)
    model = CartesianDiscreteModel(domain, (l,l))
    boundary_tags= ["boundary"]
    if dirichlet  
        dirichlet_tags= ["boundary"] 
        dirichlet_values = [0.0];
    end

elseif square_circle
    displacement = 0.0 
    h = 1.0/10.0
    p = (π, π, 0.5*π, 0.5*π + displacement, 0.15*π, h) # for square_circle (Length_x, Length_y, circle_center_x, circle_center_y, radious)
    model = GmshDiscreteModel("models/square_circle.msh")
    boundary_tags = ["circle", "ext"]
    if dirichlet 
        dirichlet_tags= ["circle","ext"] 
        dirichlet_values = [0.0,0.0]
    end 

elseif circle_circle
    h = 1.0/10.0
    p = (0.0, 0.0, 0.5*π, 1.5*π, h) # for circle_circle (cy_center_x, cy_center_y, cy_inner_radious, cy_outer_radious)
    model = GmshDiscreteModel("models/circle_circle.msh")
    boundary_tags = ["inner","outer"]
    if dirichlet 
        dirichlet_tags=["inner","outer"]
        dirichlet_values = [0.0,0.0];
    end
    

elseif open_cavity
    L=1.0*pi
    ll = 0.4
    side_x = L
    side_y = L
    small_x = ll*L
    small_y = ll*L
    openning = 0.05*L
    thikness = 0.1*L
    model = GmshDiscreteModel("models/cavity_box.msh") 
    boundary_tags= ["int","ext"]
    if dirichlet 
        dirichlet_tags= ["int","ext"] 
        dirichlet_values = [0.0,0.0];
    end 
end

The following cell is for creating meshes, we recommend to create them directly with Gmsh and then import them here as done above. 

In [ ]:
#=
if load_mesh = false

if test: 
    mesh = RectangleMesh(Point(0, 0), Point(side_x, side_y), 100, 100)
if rectangle:
    mesh = RectangleMesh(Point(0, 0), Point(side_x, side_y), 20, 20)
if open_cavity:
    big_square = Rectangle(Point(0., 0.), Point(side_x,side_y))
    
    triangle_vertices = [
    Point((side_x + openning)/2., (side_y - small_y)/2.),
    Point((side_x + small_x )/2., (side_y - small_y)/2.),
    Point((side_x + small_x )/2., (side_y + small_y)/2.),
    Point((side_x - small_x )/2., (side_y + small_y)/2.),
    Point((side_x - small_x )/2., (side_y - small_y)/2.),
    Point((side_x - openning)/2., (side_y - small_y)/2.),
    Point((side_x - openning)/2., (side_y - small_y + thikness)/2.),
    Point((side_x - small_x + thikness)/2., (side_y - small_y + thikness)/2.),
    Point((side_x - small_x + thikness)/2., (side_y + small_y - thikness)/2.),
    Point((side_x + small_x - thikness)/2., (side_y + small_y - thikness)/2.),
    Point((side_x + small_x - thikness)/2., (side_y - small_y + thikness)/2.),
    Point((side_x + openning)/2., (side_y - small_y + thikness)/2.)
    ]
    
    small_cavity = Polygon(triangle_vertices)
    
    domain = big_square - small_cavity
#    domain = small_cavity
    mesh = generate_mesh(domain,42)
    
coords = mesh.coordinates()
mpl.figure(figsize=(10,10))
triplot(coords[:,0], coords[:,1], triangles=mesh.cells())
#print mesh.num_cells()
show()

end

if load_mesh
    model = GmshDiscreteModel("models/$grid_type.msh")
else
    model = make_model(grid_type, p)
    save("models/$grid_type.msh", "mesh", model)
end
=#

In [ ]:
Ω = Triangulation(model)
degree = 3
dΩ = Measure(Ω,degree)

We plot the grid to check that everything is OK

In [ ]:
fig, ax = plot(Ω)
ax.aspect = AxisAspect(1) # this makes the region with fixed aspect, so it might not give the correct ratios
wireframe!(Ω, color=:black, linewidth=1)
scatter!(Ω, marker=:star8, markersize=4, color=:blue)
fig

And now plot the boundaries.

In [ ]:
Γ = BoundaryTriangulation(model,tags=boundary_tags)
dΓ = Measure(Γ,degree)
fig, ax = plot(Γ, linewidth=8)
ax.aspect = AxisAspect(1)
wireframe!(Γ, color=:black, linewidth=1)
fig

Create the Finite Element spaces. You can use higher order elements too.

In [ ]:
order = 2
reffe = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(model,reffe;vector_type=Vector{ComplexF64},conformity=:H1,dirichlet_tags = dirichlet_tags)
U = TrialFESpace(V,dirichlet_values)


Now we create the matrices of the problem. The problem is of the form:

$$ 
A*u_j = \lambda_j B*u_j 
$$
For this case the matrix $B$ is just the product of the finite elements base.
We do this by creating weak form for them and handling them to the Gridap library to produce the operators
separately. Once this is done we handle the matrices to the Arpack library, compute the eigenvectors and eigenvalues and return the vectors as finite elements objects. All this is done internally by functions defined in the file: `eigen.jl`.

In [ ]:
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ
m(u,v) = ∫(u*v)dΩ


In [ ]:
include("eigen.jl")

Now we can solve the problem. The parameters appearing in the problem, and many others, can be studied in the Arpack documentation. 

In [ ]:
sigma = 10
nev = 80 # number of eigenvalues asked to evaluate.
prob = EigenProblem(a, m, U, V; nev = nev, sigma = sigma)
ξ, uₕs = solve(prob);

We list the eigenvalues obtained:

In [ ]:
ξ

And we can plot some of the eigenvectors. 

In [ ]:
n = 70
uₕ = uₕs[n]
fig, ax, plt = plot(Ω, real(uₕ), colormap=:heat)
ax.aspect = AxisAspect(1)
Colorbar(fig[2,1], plt, vertical=false)
fig
#This plot some times does not work, in particular with meshes created with Gridap.



We can also save the individual eigenvectors to later visualize. 

In [ ]:

writevtk(Ω,"Cavity/eigvector_$n",cellfields=["u_real" => real(uₕ), "u_imag" => imag(uₕ)])


We can arrange the eigenvalues by their real parts. This also can be done directly choosing some of the EigenProblem parameters.

In [ ]:
scatter(sort(real(ξ)))

To compare with the exact results with their degeneracy.

In [ ]:
if test 
    [n^2 + m^2 for n = 1:5, m = 1:5]
end